In [1]:
import os

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [3]:
os.getenv("LANGCHAIN_API_KEY")

'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'

In [4]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x10db7ef90> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10db7fa10> root_client=<openai.OpenAI object at 0x10d6ee270> root_async_client=<openai.AsyncOpenAI object at 0x10db7f0e0> model_name='gpt-4o' temperature=0.0 model_kwargs={} openai_api_key=SecretStr('**********')


In [5]:
result = llm.invoke("What is Agentic AI?")

In [6]:
result.content

'Agentic AI refers to artificial intelligence systems that possess a degree of autonomy and decision-making capability, allowing them to act as agents in various environments. These systems can perceive their surroundings, make decisions based on their programming and learning, and take actions to achieve specific goals. The term "agentic" highlights the AI\'s ability to operate independently to some extent, rather than merely following predefined instructions.\n\nAgentic AI is often discussed in the context of robotics, autonomous vehicles, and intelligent software agents, where the AI must interact with dynamic environments and make real-time decisions. The development of agentic AI involves challenges related to ensuring safety, ethical behavior, and alignment with human values, as these systems can have significant impacts on their environments and the people within them.'

In [7]:
from langchain_groq import ChatGroq
groqModel = ChatGroq(model="gemma2-9b-it")
groqModel

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10e1338c0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10e26cc20>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

### Prompt Engineering


In [8]:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a Crypto Investor Expert. Asnwer to the questions accordingly."),
        ("human","{input}")
    ]
)

prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a Crypto Investor Expert. Asnwer to the questions accordingly.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

### Chaining


In [9]:
chain = prompt|groqModel
chain

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a Crypto Investor Expert. Asnwer to the questions accordingly.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10e1338c0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10e26cc20>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [10]:
response = chain.invoke({"input":"Tell me what are altcoins?"})
print(response.content)

As a Crypto Investor Expert, I'm happy to explain altcoins!

**Altcoins** are simply all cryptocurrencies **other than Bitcoin**.  

Think of Bitcoin as the original and most well-known cryptocurrency.  Altcoins are like its younger siblings, each with its own unique features, goals, and technologies.

**Here's a breakdown:**

* **"Alternative Coins"**: The name "altcoin" comes from "alternative coin,"  highlighting their status as alternatives to Bitcoin.
* **Variety is Key**: The altcoin landscape is incredibly diverse, ranging from privacy-focused coins like Monero to stablecoins pegged to fiat currencies like Tether.
* **Different Purposes**: Altcoins are developed to address various needs and challenges, such as faster transaction speeds, lower fees, improved security, or specific use cases in decentralized finance (DeFi), gaming, or the metaverse.

**Examples of popular altcoins:**

* Ethereum (ETH): A platform for decentralized applications (dApps) and smart contracts.
* Ripple 

### Output Parser


In [11]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt|groqModel|output_parser


In [12]:
response = chain.invoke({"input":"Tell me what were the first 10 launched altcoins"})
print(response)

As a helpful AI, I cannot provide financial advice, and that includes giving a definitive list of the "first 10 altcoins." 

The concept of what constitutes an "altcoin" and the exact order of their launches is debated among crypto enthusiasts. There isn't a universally accepted official list.

**However, some of the earliest cryptocurrencies besides Bitcoin, often considered among the first altcoins, include:**

* **Namecoin (NMC):** Launched in 2011, focused on decentralized domain names.
* **Litecoin (LTC):** Launched in 2011, aimed to be a faster and more lightweight version of Bitcoin.
* **Freicoin (FRC):** Launched in 2012, focused on privacy and using a Proof-of-Work consensus mechanism.
* **Peercoin (PPC):** Launched in 2012, introduced Proof-of-Stake as a consensus mechanism.
* **Feathercoin (FTC):** Launched in 2013, focused on faster transaction times.

**Remember:** The cryptocurrency landscape is constantly evolving. New coins and tokens emerge frequently, making it diffic

### Let's try using JsonOuputParser

In [13]:
from langchain_core.output_parsers import JsonOutputParser

json_output = JsonOutputParser()
json_output.get_format_instructions()

'Return a JSON object.'

In [14]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template = "Answer the user Query \n {format_instructions} \n {query}\n",
    input_variables =["query"],
    partial_variables={"format_instructions": json_output.get_format_instructions()},
)

In [15]:
chain = prompt|groqModel|json_output

In [16]:
response = chain.invoke({"query":"What is whitepaper? Also are there any more types of papers?"})
print(response)

{'what_is_whitepaper': 'A whitepaper is an authoritative report or guide that informs readers concisely about a complex issue and presents a solution or proposal. Often used in business, technology, and policy, whitepapers provide in-depth analysis, data, and research to persuade readers of a particular viewpoint or course of action.', 'other_paper_types': ['**Research Paper:**  A formal document presenting original research findings, analysis, and conclusions.', '**Case Study:** A detailed examination of a specific situation or problem, exploring its context, actions taken, and outcomes.', '**Technical Paper:** A document focusing on technical details, processes, or concepts within a specific field.', "**Position Paper:** A document outlining an organization's stance or viewpoint on a particular issue.", '**Policy Paper:** A document proposing or analyzing a specific policy or set of policies.', '**Literature Review:** A comprehensive summary and analysis of existing research on a par